In [ ]:
%pip install gymnasium --quiet
%pip install pettingzoo --quiet
%pip install pygame --quiet
%pip install numpy --quiet
%pip install scipy --quiet

# Custon Gym Environment

In [ ]:
import functools

import gymnasium
import numpy as np
import pygame
from gymnasium.spaces import Discrete, MultiDiscrete
from pettingzoo import ParallelEnv
from scipy.stats import levy_stable


class SurgeryQuotaScheduler(ParallelEnv):

    metadata = {'render_modes': ['human', 'terminal'],
                'name': 'sqsc_v1'}

    def __init__(self, render_mode=None, max_agents=12, max_days=7, max_episode_length=7):
        self.max_agents = max_agents
        self.max_days = max_days
        self.max_episode_length = max_episode_length
        self.possible_agents = ["agent_" + str(r) for r in range(self.max_agents)]
        self.agent_name_mapping = dict(zip(self.possible_agents, list(range(len(self.possible_agents)))))
        self.agent_action_mapping = {0: 1, 1: -1, 2: 0}
        self.render_mode = render_mode

    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        return MultiDiscrete([3, 2, 2, self.max_days, *[self.max_agents+2 for _ in range(self.max_days)]], start=[1, 0, 0, 0, *[-1 for _ in range(self.max_days)]])

    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return Discrete(3)

    def render(self):
        if self.render_mode == "human":
            pygame.init()
            win_width, win_height = 1000, 600
            win = pygame.display.set_mode((win_width, win_height))
            pygame.display.set_caption("Surgery Quota Scheduler")
            run = True
            while run:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        run = False
                win.fill((255, 255, 255))
                font = pygame.font.Font(None, 36)
                y = 50
                for agent, pos in {agent: self.agents_data[agent]['position'] for agent in self.agents}.items():
                    text = font.render(f"{agent}: Day {pos + 1}", True, (0, 0, 0))
                    win.blit(text, (50, y))
                    y += 30
                pygame.display.flip()
            pygame.quit()            
        elif self.render_mode == 'terminal':
            return self.observed_state
        else:
            gymnasium.logger.warn('You are calling render mode without specifying any render mode.')
            return

    def close(self):
        if self.render_mode == "human":
            pygame.quit()

    def reset(self, seed=None, options=None):
        self.rng = np.random.default_rng(seed)
        self.num_moves = 0
        self.agents = self.possible_agents[:]
        self.target_state = {day: (boundaries['min'] + boundaries['max']) / 2 for day, boundaries in options['target_state'].items()} if options is not None else {0: 3, 1: 2, 2: 2, 3: 2, 4: 1, 5: 1, 6: 1}
        self.agents_data = options['agents_data'] if options is not None else {agent: {'active': True,
                                                                                       'position': self.rng.integers(0, 7, 1).item(),
                                                                                       'base_reward': 1.0,
                                                                                       'window': 5,
                                                                                       'alpha': 2.0,
                                                                                       'urgency': self.rng.integers(1, 4, 1).item(),
                                                                                       'completeness': self.rng.integers(0, 2, 1).item(),
                                                                                       'complexity': self.rng.integers(0, 2, 1).item(),
                                                                                       'mutation_rate': 0.0} for agent in self.agents}
        observations = {agent: np.array([self.agents_data[agent]['urgency'],
                                         self.agents_data[agent]['completeness'],
                                         self.agents_data[agent]['complexity'],
                                         self.agents_data[agent]['position'],
                                         *[self.observed_state[day] if day in [int((self.agents_data[agent]['position'] + np.ceil(d - self.agents_data[agent]['window'] / 2)) % self.max_days) for d in range(self.agents_data[agent]['window'])] else -1 for day in self.observed_state]])
                                         for agent in self.agents}
        infos = {agent: self.agents_data[agent] for agent in self.agents}
        return observations, infos

    def step(self, actions):
        if not actions:
            return {}, {}, {}, {}, {}
        self.num_moves += 1
        for agent in self.agents:
            if self.agents_data[agent]['active']:
                self.agents_data[agent]['position'] = (self.agents_data[agent]['position'] + self.agent_action_mapping[int(actions[agent])]) % self.max_days
                if self.agents_data[agent]['position'] > (self.max_days // 2):
                    if int(actions[agent]) == 0:
                        if self.agents_data[agent]['mutation_rate'] != 1.0:
                            self.agents_data[agent]['mutation_rate'] = min(self.agents_data[agent]['mutation_rate'] + 0.05, 1.0)
                    elif int(actions[agent]) == 1:
                        if self.agents_data[agent]['mutation_rate'] != 0.0:
                            self.agents_data[agent]['mutation_rate'] = max(self.agents_data[agent]['mutation_rate'] - 0.05, 0.0)
        rewards = {agent: self.reward_map(agent) for agent in self.agents}
        terminations = {agent: list(actions.values()).count(2) > self.max_agents * 0.8 for agent in self.agents}
        truncations = {agent: self.num_moves >= self.max_episode_length for agent in self.agents}
        if any(terminations.values()) or any(truncations.values()):
            rewards = {agent: r - 0.5 * abs(self.observed_state[self.agents_data[agent]['position']] - self.target_state[self.agents_data[agent]['position']]) for agent, r in rewards.items()}
        observations = {agent: np.array([self.agents_data[agent]['urgency'],
                                         self.agents_data[agent]['completeness'],
                                         self.agents_data[agent]['complexity'],
                                         self.agents_data[agent]['position'],
                                         *[self.observed_state[day] if day in [int((self.agents_data[agent]['position'] + np.ceil(d - self.agents_data[agent]['window'] / 2)) % self.max_days) for d in range(self.agents_data[agent]['window'])] else -1 for day in self.observed_state]])
                                         for agent in self.agents}
        infos = {agent: self.agents_data[agent] for agent in self.agents}
        return observations, rewards, terminations, truncations, infos

    @property
    def observed_state(self):
        return {day: [self.agents_data[agent]['position'] for agent in self.agents].count(day) for day in range(self.max_days)}
    
    def reward_map(self, agent):
        discrepancy = {day: abs(self.observed_state[day] - self.target_state[day]) for day in self.target_state}
        window = [int((self.agents_data[agent]['position'] + np.ceil(
            d - self.agents_data[agent]['window'] / 2)) % self.max_days) for d in
                range(self.agents_data[agent]['window'])]
        masked_discrepancy = {day: discrepancy[day] if day in window else 0 for day in discrepancy}
        rv = levy_stable(self.agents_data[agent]['alpha'], 0.0, loc=self.agents_data[agent]['position'], scale=1.0)
        weighted_discrepancy_sum = np.sum([rv.pdf(day) * masked_discrepancy[day] for day in masked_discrepancy])

        global_discrepancy = sum(discrepancy.values())

        reward = - (weighted_discrepancy_sum + 0.05 * global_discrepancy) * self.agents_data[agent]['base_reward'] / 2

        current_day = self.agents_data[agent]['position']
        if self.observed_state[current_day] < self.target_state[current_day]:
            reward += 0.5 * self.agents_data[agent]['base_reward']

        if self.observed_state[current_day] > self.target_state[current_day]:
            reward -= 0.1 * self.agents_data[agent]['base_reward']

        scale = max(1, (self.agents_data[agent]['complexity'] + (1 - self.agents_data[agent]['completeness'])) * self.agents_data[agent]['urgency'])
        if scale > 3:
            reward -= (self.agents_data[agent]['position'] - 1) / 5

        return reward


In [ ]:
%pip install torch --quiet
%pip install tqdm --quiet

# MAPPO trainer

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from torch.utils.tensorboard import SummaryWriter
import os
from tqdm import tqdm


class Actor(nn.Module):
    def __init__(self, obs_dim, action_dim):
        super(Actor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(obs_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim),
            nn.Softmax(dim=-1)
        )
    
    def forward(self, obs):
        return self.network(obs)


class Critic(nn.Module):
    def __init__(self, obs_dim):
        super(Critic, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(obs_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    
    def forward(self, obs):
        return self.network(obs)


class MAPPOAgent:
    def __init__(self, obs_dim, action_dim, lr_actor=3e-4, lr_critic=3e-4, gamma=0.99, epsilon=0.2):
        self.actor = Actor(obs_dim, action_dim)
        self.critic = Critic(obs_dim)
        self.optimizer_actor = optim.Adam(self.actor.parameters(), lr=lr_actor)
        self.optimizer_critic = optim.Adam(self.critic.parameters(), lr=lr_critic)
        self.gamma = gamma
        self.epsilon = epsilon
    
    def get_action(self, obs):
        obs = torch.FloatTensor(obs)
        probs = self.actor(obs)
        dist = Categorical(probs)
        action = dist.sample()
        return action.item(), dist.log_prob(action)
    
    def update(self, obs, actions, old_log_probs, rewards, next_obs, dones):
        obs = torch.FloatTensor(obs)
        actions = torch.LongTensor(actions)
        old_log_probs = torch.FloatTensor(old_log_probs)
        rewards = torch.FloatTensor(rewards)
        next_obs = torch.FloatTensor(next_obs)
        dones = torch.FloatTensor(dones)
        
        values = self.critic(obs).squeeze()
        next_values = self.critic(next_obs).squeeze()
        td_target = rewards + self.gamma * next_values * (1 - dones)
        td_error = td_target - values
        advantage = td_error.detach()
        
        critic_loss = (td_error ** 2).mean()
        self.optimizer_critic.zero_grad()
        critic_loss.backward()
        self.optimizer_critic.step()
        
        probs = self.actor(obs)
        dist = Categorical(probs)
        new_log_probs = dist.log_prob(actions)
        ratio = torch.exp(new_log_probs - old_log_probs)
        surr1 = ratio * advantage
        surr2 = torch.clamp(ratio, 1 - self.epsilon, 1 + self.epsilon) * advantage
        actor_loss = -torch.min(surr1, surr2).mean()
        self.optimizer_actor.zero_grad()
        actor_loss.backward()
        self.optimizer_actor.step()
        
        return critic_loss.item(), actor_loss.item()


class MAPPOTrainer:
    def __init__(self, env, n_agents, obs_dim, action_dim, lr_actor=3e-4, lr_critic=3e-4, gamma=0.99, epsilon=0.2):
        self.env = env
        self.n_agents = n_agents
        self.agents = [MAPPOAgent(obs_dim, action_dim, lr_actor, lr_critic, gamma, epsilon) for _ in range(n_agents)]
        self.writer = SummaryWriter()
    
    def train(self, n_episodes, max_steps, log_interval=5):
        pbar = tqdm(total=n_episodes, desc="Training Progress")
        for episode in range(n_episodes):
            obs, _ = self.env.reset()
            episode_rewards = [0 for _ in range(self.n_agents)]
            episode_critic_losses = []
            episode_actor_losses = []
            
            for step in range(max_steps):
                actions = []
                old_log_probs = []
                
                for i, agent in enumerate(self.agents):
                    action, log_prob = agent.get_action(obs[f"agent_{i}"])
                    actions.append(action)
                    old_log_probs.append(log_prob)
                
                next_obs, rewards, dones, truncations, _ = self.env.step({f"agent_{i}": a for i, a in enumerate(actions)})
                
                for i in range(self.n_agents):
                    episode_rewards[i] += rewards[f"agent_{i}"]
                
                for i, agent in enumerate(self.agents):
                    critic_loss, actor_loss = agent.update(
                        obs[f"agent_{i}"].reshape(1, -1),
                        [actions[i]],
                        [old_log_probs[i].item()],
                        [rewards[f"agent_{i}"]],
                        next_obs[f"agent_{i}"].reshape(1, -1),
                        [dones[f"agent_{i}"] or truncations[f"agent_{i}"]]
                    )
                    episode_critic_losses.append(critic_loss)
                    episode_actor_losses.append(actor_loss)
                
                obs = next_obs
                
                if all(dones.values()) or all(truncations.values()):
                    break
            
            if (episode + 1) % log_interval == 0:
                avg_reward = sum(episode_rewards) / self.n_agents
                avg_critic_loss = np.mean(episode_critic_losses)
                avg_actor_loss = np.mean(episode_actor_losses)
                self.writer.add_scalar('Reward/average', avg_reward, episode)
                self.writer.add_scalar('Loss/critic', avg_critic_loss, episode)
                self.writer.add_scalar('Loss/actor', avg_actor_loss, episode)
            
            pbar.update(1)
            pbar.set_postfix({'avg_reward': f'{sum(episode_rewards) / self.n_agents:.2f}'})
        
        pbar.close()
    
    def save_model(self, path):
        if not os.path.exists(path):
            os.makedirs(path)
        for i, agent in enumerate(self.agents):
            torch.save(agent.actor.state_dict(), os.path.join(path, f'actor_{i}.pth'))
            torch.save(agent.critic.state_dict(), os.path.join(path, f'critic_{i}.pth'))


if __name__ == "__main__":
    env = SurgeryQuotaScheduler()
    n_agents = 12
    obs_dim = env.observation_space("agent_0").shape[0]
    action_dim = env.action_space("agent_0").n
    
    trainer = MAPPOTrainer(env, n_agents, obs_dim, action_dim)
    trainer.train(n_episodes=12000, max_steps=7, log_interval=10)
    trainer.save_model('trained_model')


# MAPPO tester

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.distributions import Categorical
from tqdm import tqdm


class Actor(nn.Module):
    def __init__(self, obs_dim, action_dim):
        super(Actor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(obs_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim),
            nn.Softmax(dim=-1)
        )
    
    def forward(self, obs):
        return self.network(obs)


class MAPPOAgent:
    def __init__(self, obs_dim, action_dim):
        self.actor = Actor(obs_dim, action_dim)
    
    def load_model(self, path):
        self.actor.load_state_dict(torch.load(path))
        self.actor.eval()
    
    def get_action(self, obs):
        obs = torch.FloatTensor(obs)
        with torch.no_grad():
            probs = self.actor(obs)
        dist = Categorical(probs)
        action = dist.sample()
        return action.item()


class MAPPOTester:

    def __init__(self, env, n_agents, obs_dim, action_dim, options=None):
        self.env = env
        self.options = options
        self.n_agents = n_agents
        self.agents = [MAPPOAgent(obs_dim, action_dim) for _ in range(n_agents)]

    def calculate_deviation(self, observed_states, target_state):
        num_episodes = len(observed_states)
        bootstrap_average_distribution = {i: sum(episode[i] for episode in observed_states) / num_episodes for i in
                                          range(7)}

        bootstrap_average_percentage_deviations = {i: 0 for i in range(7)}
        mean_target_state = {day: (target_state[day]['max'] + target_state[day]['min']) / 2 for day in target_state}
        for key, value in bootstrap_average_distribution.items():
            bootstrap_average_percentage_deviations[key] = np.abs(mean_target_state[key] - value) / mean_target_state[key]
        
        average_bootstrap_deviation = np.mean(list(bootstrap_average_percentage_deviations.values()))
        std_bootstrap_deviation = np.std(list(bootstrap_average_percentage_deviations.values()))
        return average_bootstrap_deviation, std_bootstrap_deviation

    def test(self, n_episodes, max_steps, target_state):
        all_observed_states = []
        all_final_positions = []
        all_scaling_factors = []

        for e in tqdm(range(n_episodes), desc="Bootstrap testing..."):
            obs, info = self.env.reset(options=self.options)

            # print(f'Episode {e}:')
            # print(f'Initial environment state', self.env.render())
            # print('Beliefs: ', obs)
            episode_observed_states = []

            for step in range(max_steps):
                actions = {}
                for i, agent in enumerate(self.agents):
                    action = agent.get_action(obs[f"agent_{i}"])
                    actions[f"agent_{i}"] = action
                # print('Intentions: ', actions)
                next_obs, rewards, dones, truncations, info = self.env.step(actions)
                # print('Desires: ', rewards)
                # print(f'Environment state on step {step}:', self.env.render())
                # print('Beliefs: ', next_obs)
                
                if any(dones.values()) or any(truncations.values()):
                    episode_observed_states.append(self.env.observed_state)
                    all_final_positions.extend([agent_info['position'] for agent_info in info.values()])
                    all_scaling_factors.extend([max(1, (agent_info['complexity'] + (1 - agent_info['completeness'])) * agent_info['urgency'])  for agent_info in info.values()])
                    # all_scaling_factors.extend([agent_info['scaling_factor'] for agent_info in info.values()])

                obs = next_obs

                if all(dones.values()) or all(truncations.values()):
                    break

            all_observed_states.extend(episode_observed_states)

        mean_deviation, std_deviation = self.calculate_deviation(all_observed_states, target_state)
        avg_bids_per_day = {day: sum(state[day] for state in all_observed_states) / len(all_observed_states) for day in
                            range(7)}

        scaling_factor_positions = {}
        for sf, pos in zip(all_scaling_factors, all_final_positions):
            if sf not in scaling_factor_positions:
                scaling_factor_positions[sf] = []
            scaling_factor_positions[sf].append(pos)
        avg_position_per_scaling_factor = {sf: np.mean(positions) for sf, positions in scaling_factor_positions.items()}

        return mean_deviation, std_deviation, avg_bids_per_day, avg_position_per_scaling_factor

    def bootstrap_test(self, n_episodes, max_steps, target_state):
        mean_deviation, std_deviation, avg_bids, avg_positions = self.test(n_episodes, max_steps, target_state)

        print(f"Operator preferences: {target_state}")
        print(f"Average number of bids per day: {avg_bids}")
        print(f"Percentage deviation: mean = {mean_deviation:.2%}, std = {std_deviation: .2%}")
        print(f"Average position per scaling factor: {avg_positions}")
    
    def load_model(self, path):
        for i, agent in enumerate(self.agents):
            agent.load_model(os.path.join(path, f'actor_{i}.pth'))
        print(f"Loaded model from {path}")


if __name__ == "__main__":
    env = SurgeryQuotaScheduler(render_mode='terminal')
    n_agents = 12
    obs_dim = env.observation_space("agent_0").shape[0]
    action_dim = env.action_space("agent_0").n
    
    tester = MAPPOTester(env, n_agents, obs_dim, action_dim)
    tester.load_model(path='trained_model')
    tester.bootstrap_test(n_episodes=10000, max_steps=7,
                          target_state={0: {'min': 3, 'max': 3},
                                        1: {'min': 2, 'max': 2},
                                        2: {'min': 2, 'max': 2},
                                        3: {'min': 2, 'max': 2},
                                        4: {'min': 1, 'max': 1},
                                        5: {'min': 1, 'max': 1},
                                        6: {'min': 1, 'max': 1}})


# Re-Scheduling

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.distributions import Categorical
import itertools
import logging


class Actor(nn.Module):
    def __init__(self, obs_dim, action_dim):
        super(Actor, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(obs_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim),
            nn.Softmax(dim=-1)
        )
    
    def forward(self, obs):
        return self.network(obs)


class MAPPOAgent:
    def __init__(self, obs_dim, action_dim):
        self.actor = Actor(obs_dim, action_dim)
    
    def load_model(self, path):
        self.actor.load_state_dict(torch.load(path, weights_only=True))
        self.actor.eval()
    
    def get_action(self, obs):
        obs = torch.FloatTensor(obs)
        with torch.no_grad():
            probs = self.actor(obs)
        dist = Categorical(probs)
        action = dist.sample()
        return action.item()


class Client:
    def __init__(self, name, urgency, completeness, complexity) -> None:
        self.name = name
        self.urgency = urgency
        self.completeness = completeness
        self.complexity = complexity
        self.acceptance_rate = np.random.randint(50, 76) / 100
        self._day = np.random.randint(0, 7)
        self._satisfied = False
        self._assigned_agent = None

    @property
    def appointment_day(self):
        return self._day

    @appointment_day.setter
    def appointment_day(self, value):
        self._day = value

    @property
    def satisfied(self):
        return self._satisfied

    @satisfied.setter
    def satisfied(self, value):
        self._satisfied = value

    @property
    def assigned_agent(self):
        return self._assigned_agent

    @assigned_agent.setter
    def assigned_agent(self, value):
        self._assigned_agent = value

    def give_feedback(self):
        answer = np.random.choice([True, False], p=[self.acceptance_rate, 1 - self.acceptance_rate])
        if answer == True: 
            self.acceptance_rate = 1.0
        return answer
    

class MultiAgentSystemOperator:
    def __init__(self, list_of_clients) -> None:
        self.clients = list_of_clients
    
    def collect_feedback(self):
        for client in self.clients:
            client.satisfied = client.give_feedback()
    
    def assign_agnets(self, agents):
        for client in self.clients:
            health_state = (client.urgency, client.completeness, client.complexity)
            client.assigned_agent = agents[health_state]

    def get_actions(self, observaions):
        actions = {}
        for agent, client in zip(observations, self.clients):
            model = client.assigned_agent['model_file']      
            actions[agent] = model.get_action(observaions[agent])
        return actions


if __name__ == '__main__':

    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s',
                        handlers=[
                            logging.FileHandler("logfile.log"),
                            logging.StreamHandler()
                        ])
    logger = logging.getLogger()


    urgency = range(1, 4)
    completeness = range(0, 2)
    complexity = range(0, 2)

    options = list(itertools.product(urgency, completeness, complexity))

    selected_states = np.random.choice(len(options), size=12, replace=False)
    health_states  = [options[i] for i in selected_states]

    patiens = [Client(name=f'client_{i}',
                      urgency=np.random.randint(1, 4),
                      completeness=np.random.randint(0, 2), 
                      complexity=np.random.randint(0, 2)) for i in range(100)]

    agents = {f'agent_{i}': MAPPOAgent(obs_dim=11, action_dim=3) for i in range(len(health_states))}

    for i, agent in enumerate(agents):
        agents[agent].load_model(f'trained_model/actor_{i}.pth')

    manager = MultiAgentSystemOperator(list_of_clients=patiens)
    manager.assign_agnets({health_state: {'agent_name': agent_name, 'model_file': model_file} for health_state, (agent_name, model_file) in zip(health_states, agents.items())})

    e = 0

    while not all([client.satisfied for client in manager.clients]):

        logger.info(f"Starting episode {e}")

        env = SurgeryQuotaScheduler(render_mode='terminal', max_agents=len(patiens),
                                    max_days=7, max_episode_length=7)
        observations, _ = env.reset(
            options={
                'target_state': {0: {'min': 3, 'max': 3},
                                1: {'min': 2, 'max': 2},
                                2: {'min': 2, 'max': 2},
                                3: {'min': 2, 'max': 2},
                                4: {'min': 1, 'max': 1},
                                5: {'min': 1, 'max': 1},
                                6: {'min': 1, 'max': 1}},
                'agents_data': {f'agent_{i}': {'active': ~client.satisfied,
                                               'position': client.appointment_day if client.satisfied else np.random.randint(0, 7),
                                               'base_reward': 1.0,
                                               'window': 5,
                                               'alpha': 2.0,
                                               'urgency': client.urgency,
                                               'completeness': client.completeness,
                                               'complexity': client.complexity,
                                               'mutation_rate': 0.0}
                                for i, client in enumerate(manager.clients)
                                }
            }
        )

        logger.info(f"Episode {e} - {env.render()}")

        while True:
            actions = manager.get_actions(observaions=observations)
            logger.debug(f"Actions: {actions}")
            observations, _, dones, truncations, _ = env.step(actions)

            for agent, client in zip(observations, manager.clients):
                 client.appointment_day = observations[agent][3]

            logger.info(f"Episode {e} - {env.render()}")

            if any(dones.values()) or any(truncations.values()):
                break

        env.close()

        manager.collect_feedback()

        e += 1


In [ ]:
%pip install tensorflow --quiet
%pip install tensorboard --quiet

# For Comparison: Naive Genetic Algorithm

In [ ]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from scipy.stats import levy_stable
import itertools
from collections import defaultdict

class GeneticAlgorithmScheduler:
    def __init__(self, max_agents=12, max_days=7, population_size=100, generations=1000):
        self.max_agents = max_agents
        self.max_days = max_days
        self.population_size = population_size
        self.generations = generations
        self.agents_data = self.generate_agents_data()
        self.target_state = {day: self.max_agents / self.max_days for day in range(self.max_days)}
        
        self.log_dir = 'logs/genetic_algorithm'
        self.summary_writer = tf.summary.create_file_writer(self.log_dir)

    def generate_agents_data(self):
        agents_data = {}
        rng = np.random.default_rng()
        for (urgency, complexity, completeness), agent in zip(
            itertools.product(range(1, 4), range(0, 2), range(0, 2)),
            [f"agent_{r}" for r in range(self.max_agents)]
        ):
            agents_data[agent] = {
                'active': True,
                'base_reward': 1.0,
                'window': 3,
                'alpha': 2.0,
                'urgency': urgency,
                'complexity': complexity,
                'completeness': completeness,
                'mutation_rate': 0.0,
                'position': rng.integers(0, 7, 1).item(),
                'scaling_factor': ((complexity + (1 - completeness)) * urgency)
            }
        return agents_data

    def create_individual(self):
        return np.array([self.agents_data[f"agent_{i}"]["position"] for i in range(self.max_agents)])

    def create_population(self):
        return [self.create_individual() for _ in range(self.population_size)]

    def fitness(self, individual):
        observed_state = {day: 0 for day in range(self.max_days)}
        for agent, day in enumerate(individual):
            observed_state[day] += 1
        
        fitness_score = 0
        for agent, day in enumerate(individual):
            agent_data = self.agents_data[f"agent_{agent}"]
            discrepancy = {d: abs(observed_state[d] - self.target_state[d]) for d in range(self.max_days)}
            window = [int((day + np.ceil(d - agent_data['window'] / 2)) % self.max_days) for d in range(agent_data['window'])]
            masked_discrepancy = {d: discrepancy[d] if d in window else 0 for d in discrepancy}
            rv = levy_stable(agent_data['alpha'], 0.0, loc=day, scale=1.0)
            weighted_discrepancy_sum = np.sum([rv.pdf(d) * masked_discrepancy[d] for d in masked_discrepancy])
            reward = -weighted_discrepancy_sum * agent_data['base_reward']
            fitness_score += reward
        
        return fitness_score

    def selection(self, population, k=2):
        selected = np.random.choice(len(population), k, replace=False)
        return max([population[i] for i in selected], key=self.fitness)

    def crossover(self, parent1, parent2):
        crossover_point = np.random.randint(1, len(parent1))
        child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        return child

    def mutation(self, individual, mutation_rate=0.01):
        for i in range(len(individual)):
            if np.random.random() < mutation_rate:
                individual[i] = np.random.randint(0, self.max_days)
        return individual

    def run(self):
        population = self.create_population()
        best_fitness_history = []
        
        for generation in tqdm(range(self.generations), leave=False):
            new_population = []
            
            for _ in range(self.population_size):
                parent1 = self.selection(population)
                parent2 = self.selection(population)
                child = self.crossover(parent1, parent2)
                child = self.mutation(child)
                new_population.append(child)
            
            population = new_population
            
            best_individual = max(population, key=self.fitness)
            best_fitness = self.fitness(best_individual)
            best_fitness_history.append(best_fitness)
            
            with self.summary_writer.as_default():
                tf.summary.scalar('Best Fitness', best_fitness, step=generation)
        
        return max(population, key=self.fitness), best_fitness_history

    def get_schedule_details(self, best_individual):
        schedule_details = []
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        
        for agent, day in enumerate(best_individual):
            agent_data = self.agents_data[f"agent_{agent}"]
            schedule_details.append({
                'agent': f"agent_{agent}",
                'day': days[day],
                'urgency': agent_data['urgency'],
                'completeness': agent_data['completeness'],
                'complexity': agent_data['complexity'],
                'scaling_factor': agent_data['scaling_factor']
            })
        
        return schedule_details

def run_experiment(num_runs=5):
    all_results = []
    all_fitness_histories = []

    for run in range(num_runs):
        print(f"\nRun {run + 1}/{num_runs}")
        scheduler = GeneticAlgorithmScheduler()
        best_schedule, fitness_history = scheduler.run()
        schedule_details = scheduler.get_schedule_details(best_schedule)
        all_results.append(schedule_details)
        all_fitness_histories.append(fitness_history)

        print(f"Best fitness for run {run + 1}: {fitness_history[-1]}")
        for item in schedule_details:
            print(f"{item['agent']} placed on {item['day']}, urgency: {item['urgency']}, "
                  f"completeness: {item['completeness']}, complexity: {item['complexity']}, "
                  f"scaling_factor: {item['scaling_factor']}")

    return all_results, all_fitness_histories

def summarize_results(all_results, all_fitness_histories):
    num_runs = len(all_results)
    
    final_fitness_scores = [history[-1] for history in all_fitness_histories]
    avg_final_fitness = np.mean(final_fitness_scores)
    std_final_fitness = np.std(final_fitness_scores)
    
    print(f"\nSummary across {num_runs} runs:")
    print(f"Average final fitness: {avg_final_fitness:.2f} ± {std_final_fitness:.2f}")

    day_counts = defaultdict(lambda: defaultdict(int))
    for run_results in all_results:
        for item in run_results:
            day_counts[item['agent']][item['day']] += 1
    
    print("\nMost common day assignments:")
    for agent in day_counts:
        most_common_day = max(day_counts[agent], key=day_counts[agent].get)
        frequency = day_counts[agent][most_common_day]
        print(f"{agent}: {most_common_day} ({frequency}/{num_runs} runs)")

    convergence_points = []
    for history in all_fitness_histories:
        for i, fitness in enumerate(history):
            if i > 0 and abs(fitness - history[-1]) < 0.01 * abs(history[-1]):
                convergence_points.append(i)
                break
    avg_convergence = np.mean(convergence_points)
    print(f"\nAverage convergence generation: {avg_convergence:.2f}")

if __name__ == "__main__":
    all_results, all_fitness_histories = run_experiment(num_runs=5)
    summarize_results(all_results, all_fitness_histories)


# For comparison: Expert Heuristic

In [ ]:

import itertools
import numpy as np

class Client:
    def __init__(self, name, urgency, completeness, complexity):
        self.name = name
        self.urgency = urgency
        self.completeness = completeness
        self.complexity = complexity

    @property
    def scaling_factor(self):
        return int(round((self.complexity + (1 - self.completeness)) * self.urgency))

class Manager:
    def __init__(self, planning_horizon, preferences):
        self.planning_horizon = planning_horizon
        self.preferences = preferences if preferences is not None else {
            day: {'min': np.min(numbers), 'max': np.max(numbers)}
            for day, numbers in enumerate(np.random.randint(1, 3, (7, 2)))
        }

    def reset_schedule(self):
        self.schedule = {day: [] for day in range(self.planning_horizon)}

    def manage(self, upcoming_requests):
        for request in upcoming_requests:
            for day in range(self.planning_horizon):
                if len(self.schedule[day]) < self.preferences[day]['max']:
                    self.schedule[day].append(request)
                    break
        return self.schedule

class Estimator:
    def __init__(self, schedules, target_state=None):
        self.observed_states = [{day: len(schedule[day]) for day in schedule.keys()} for schedule in schedules]
        self.schedules = schedules
        if target_state is not None:
            self.target_state = target_state
        else:
            self.target_state = {
                day: {'min': np.min(numbers), 'max': np.max(numbers)}
                for day, numbers in enumerate(np.random.randint(1, 3, (7, 2)))
            }

    def describe(self):
        num_episodes = len(self.observed_states)

        deviations = []
        total_clients_per_day = {i: 0 for i in range(len(self.schedules[0]))}
        all_scaling_factors = []
        all_final_positions = []

        for schedule in self.schedules:
            for day in schedule:
                actual_clients = len(schedule[day])
                total_clients_per_day[day] += actual_clients
                target_min = self.target_state[day]['min']
                target_max = self.target_state[day]['max']

                deviation = max(0, target_min - actual_clients) + max(0, actual_clients - target_max)
                deviations.append(deviation)

                for client in schedule[day]:
                    all_scaling_factors.append(client.scaling_factor)
                    all_final_positions.append(day)

        mean_deviation = np.mean(deviations) / num_episodes
        std_deviation = np.std(deviations) / num_episodes
        avg_clients_per_day = {day: total / num_episodes for day, total in total_clients_per_day.items()}

        scaling_factor_positions = {}
        for sf, pos in zip(all_scaling_factors, all_final_positions):
            if sf not in scaling_factor_positions:
                scaling_factor_positions[sf] = []
            scaling_factor_positions[sf].append(pos)
        avg_position_per_scaling_factor = {sf: np.mean(positions) for sf, positions in scaling_factor_positions.items()}

        print(f"Operator preferences: {self.target_state}")
        print(f"Average number of bids per day: {avg_clients_per_day}")
        print(f"Percentage deviation: mean = {mean_deviation:.2%}, std = {std_deviation: .2%}")
        print(f"Average position per scaling factor: {avg_position_per_scaling_factor}")

def run_simulation(n_episodes=1):
    planning_horizon = 7
    health_state = list(itertools.product(range(1, 4),
                                     range(0, 2),
                                     range(0, 2)))

    target_state = {
        0: {'min': 3, 'max': 3},
        1: {'min': 2, 'max': 2},
        2: {'min': 2, 'max': 2},
        3: {'min': 2, 'max': 2},
        4: {'min': 1, 'max': 1},
        5: {'min': 1, 'max': 1},
        6: {'min': 1, 'max': 1},
    }

    manager = Manager(planning_horizon, preferences=target_state)
    
    schedules = []

    for _ in range(n_episodes):
        manager.reset_schedule()
        clients = [Client(name=f'client_{i}', urgency=urgency, completeness=completeness, complexity=complexity)
                   for i, (urgency, completeness, complexity) in enumerate(health_state)]
        schedules.append(manager.manage(np.random.choice(clients, size=12, replace=True)))

    estimator = Estimator(schedules, target_state)
    estimator.describe()

if __name__ == "__main__":
    run_simulation()
